# Spatial functions that create new geometries

Constructive methods

## Buffering

Buffering is a very common method in spatial analysis.  If you are taking this course I assume that you know what a buffer is.  In this lesson we will be creating buffers for most of the layers that we have been looking at.

In [ ]:
%matplotlib inline
import geopandas as gpd

buowl = gpd.read_file("data/BUOWL_Habitat.shp")
raptors = gpd.read_file("data/Raptor_Nests.shp")
linears = gpd.read_file("data/Linear_Projects.shp")
eagles = gpd.read_file("data/BAEA_Nests.shp")

Lets start with eagles as it is quite easy.  One issue, however, is that GeoPandas requires the distance to be in the same units as the CRS so we will want to reproject the points to UTM so we can provide the buffer distance in meters.  We will create buffers of 1/2 mile (804.5 meters) around all the eagle nests.

In [ ]:
eagles=eagles.to_crs(epsg=26913)
eagles['buffer']=eagles['geometry'].buffer(804.5)
eagles.head()

We can see that we do have a new column called buffer that contains a polygon rather than a point so it looks like it worked.  Lets plot out the buffers, but now that we have two GeoDataSeries in our GeoDataFrame we have to specify which we want to use by default using the set_geometry method.

In [ ]:
eagles.set_geometry('buffer', inplace=True)
eagles.plot()

Its hard to see if it worked or not, but it looks like the nests are smaller than the point marker so its probable that we sre seeing polygons. They just appear small at this scale.  Let's zoom in a bit.

In [ ]:
eagles.cx[500000:520000,4420000:4440000].plot()

Now lets take a look at the linears table.  We will have to convert it to UTM coordinates as well so we can use meters in our buffer distance.

In [ ]:
linears.to_crs(epsg=26913, inplace=True)
linears.tail()

In [ ]:
linears.cx[516000:520000, 4437000:4441000].plot(column='type', legend=True, figsize=(10,10))

We can see that in this area we have a number of linear projects.  These are pipelines, electric lines, access roads, etc associated with an oil and gas field in a region called the DJ Basin north of Denver, Colorado. The other layers represent wildlife species that are present and may impose environmental constraints on construction operations when they are present.  Each linear project has a right-of-way around it and it is assumed that the right away is the area of impact and so distance must be measured from it rather than from the line itself.  Thus we need to create right-of-way polygons by buffering the linear projects by the appropriate diatance.

This time we will pass the row_width column to the buffer function rather than a single distance so each polygon will be created as a buffer of the distance given in the row_width column.

In [ ]:
linears['buffer'] = linears['geometry'].buffer(linears['row_width'])
linears.head()

In [ ]:
linears.set_geometry('buffer', inplace=True)
linears.cx[516000:520000, 4437000:4441000].plot(column='type', legend=True, figsize=(10,10))

It looks like we have polygon buffers but can we show that the distance are correct?

In [ ]:
linears[linears['Project']==67].plot(figsize=(10,10))

In [ ]:
linears[linears['Project']==67]

That looks like the buffer is about 100m as it should be with a 50m row_width

In [ ]:
linears[linears['Project']==1105].plot(figsize=(10,10))

In [ ]:
linears[linears['Project']==1105]

And that looks like its 40m wide as it should be for a 20m row_width.

## Convex Hull

Convex hulls create a minimum convex polygon around a geometry.  We'll assign these to a GeoSeries named 'mcp'

In [ ]:
linears['mcp'] = linears['geometry'].convex_hull
linears.set_geometry('mcp', inplace=True)
linears.cx[516000:520000, 4437000:4441000].plot(column='type', legend=True, figsize=(10,10))

## Envelope

The envelope method creates a bounding box for each geometry. We'll store this in a field called bb.

In [ ]:
linears['bb'] = linears['geometry'].envelope
linears.set_geometry('bb', inplace=True)
linears.cx[516000:520000, 4437000:4441000].plot(column='type', legend=True, figsize=(10,10))

Now lets create buffers of 300m around the burrowing owl habitat.

In [ ]:
buowl.to_crs(epsg=26913, inplace=True)

In [ ]:
buowl['buffer']=buowl['geometry'].buffer(300)

In [ ]:
buowl.set_geometry('buffer').plot(figsize=(10,10))

## Centroid

We can also use the centroid method to create a point at the geometric center of the polygon.  **Note:** This centroid is not guaranteed to actually be inside the polygon.  If the polygon is simple it usually will be but for more complex polygons it might not be.

We will store the burrowing owl habitat's centroid in a column named centroid

In [ ]:
buowl['centroid']=buowl['geometry'].centroid
buowl.head()

In [ ]:
buowl.set_geometry('centroid').plot(figsize=(10,10))

There are more methods that create geometry but these are the most important ones.  You can see more information in the [GeoPandas](https://geopandas.org/geometric_manipulations.html) and [Shapely](https://shapely.readthedocs.io/en/latest/manual.html#constructive-methods) online documentation. 